In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
from arch import arch_model
from scipy.stats import norm

def bs_call_price(S, K, T, r, sigma):
    S = float(S)
    K = float(K)
    T = float(T)
    sigma = float(sigma)
    if T <= 0:
        return max(S - K, 0.0)
    if sigma <= 0:
        return max(S - K * np.exp(-r * T), 0.0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

ticker = "AAPL"
start = "2015-01-01"
end = "2025-01-01"

data = yf.download(ticker, start=start, end=end)
data["return"] = np.log(data["Close"]).diff()
data = data.dropna()

warmup = 50
returns = (data["return"].values * 100)
vols = [np.nan] * warmup

for t in range(warmup, len(returns)):
    am = arch_model(returns[:t], mean="Zero", vol="GARCH", p=1, q=1)
    res = am.fit(disp="off")
    f = res.forecast(horizon=1, reindex=False).variance.values[-1][0]
    vols.append(np.sqrt(f / 10000.0))

data = data.iloc[warmup:].copy()
data["forecast_vol"] = vols[warmup:]

ticker_obj = yf.Ticker(ticker)
options_dates = list(ticker_obj.options)
if len(options_dates) == 0:
    raise Exception("No options available for ticker")
options_dates_dt = [pd.to_datetime(d) for d in options_dates]

r = 0.04
days_per_year = 252
capital = 100000.0
position = 0
entry_price = 0
entry_date = None
chain_cache = {}
results = []

for i, row in data.iterrows():
    S = float(row["Close"])
    sigma = float(row["forecast_vol"])
    if np.isnan(sigma):
        continue

    future_idxs = [j for j, dt_ in enumerate(options_dates_dt) if dt_ > i]
    if not future_idxs:
        continue
    exp_idx = future_idxs[0]
    expiry_str = options_dates[exp_idx]
    expiry_dt = options_dates_dt[exp_idx]
    T = max((expiry_dt - i).days / days_per_year, 1e-6)

    if expiry_str in chain_cache:
        chain = chain_cache[expiry_str]
    else:
        try:
            chain = ticker_obj.option_chain(expiry_str)
            chain_cache[expiry_str] = chain
        except Exception:
            continue

    calls = chain.calls
    if calls.empty:
        continue

    for _, opt_row in calls.iterrows():
        mp = opt_row.get("lastPrice", np.nan)
        if isinstance(mp, pd.Series):
            mp = mp.iloc[0]

        if pd.isna(mp):
            bid = opt_row.get("bid", np.nan)
            ask = opt_row.get("ask", np.nan)
            if isinstance(bid, pd.Series):
                bid = bid.iloc[0]
            if isinstance(ask, pd.Series):
                ask = ask.iloc[0]

            if pd.notna(bid) and pd.notna(ask):
                mp = 0.5 * (bid + ask)
            elif pd.notna(ask):
                mp = ask
            elif pd.notna(bid):
                mp = bid
            else:
                continue

        market_price = float(mp)
        K = float(opt_row["strike"])
        model_price = bs_call_price(S, K, T, r, sigma)
        misprice = model_price - market_price

        if position == 0:
            if misprice > 1.0:
                position = 1
                entry_price = market_price
                entry_date = i
            elif misprice < -1.0:
                position = -1
                entry_price = market_price
                entry_date = i
        else:
            exit_flag = False
            if position == 1 and market_price >= entry_price * 1.05:
                exit_flag = True
            if position == -1 and market_price <= entry_price * 0.95:
                exit_flag = True
            if exit_flag:
                pnl = (market_price - entry_price) * position
                capital += pnl
                results.append({
                    "entry": entry_date,
                    "exit": i,
                    "pnl": pnl,
                    "capital": capital
                })
                position = 0
                entry_price = 0
                entry_date = None

bt = pd.DataFrame(results)
print(bt)
print("Final capital:", capital)


/var/folders/vb/m9plnt0538gc2wb4l5pzcgr80000gp/T/ipykernel_9098/4080900513.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
/var/folders/vb/m9plnt0538gc2wb4l5pzcgr80000gp/T/ipykernel_9098/4080900513.py:57: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S = float(row["Close"])
/var/folders/vb/m9plnt0538gc2wb4l5pzcgr80000gp/T/ipykernel_9098/4080900513.py:58: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma = float(row["forecast_vol"])
/var/folders/vb/m9plnt0538gc2wb4l5pzcgr80000gp/T/ipykernel_9098/4080900513.py:57: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser

           entry       exit    pnl    capital
0     2015-03-18 2015-03-18  10.15  100010.15
1     2015-03-18 2015-03-18   9.96  100020.11
2     2015-03-18 2015-03-18   9.50  100029.61
3     2015-03-18 2015-03-18  10.46  100040.07
4     2015-03-18 2015-03-18   9.39  100049.46
...          ...        ...    ...        ...
46827 2024-12-31 2024-12-31   3.48  355111.29
46828 2024-12-31 2024-12-31   1.87  355113.16
46829 2024-12-31 2024-12-31   2.13  355115.29
46830 2024-12-31 2024-12-31   1.72  355117.01
46831 2024-12-31 2024-12-31   1.12  355118.13

[46832 rows x 4 columns]
Final capital: 355118.1299999433


/var/folders/vb/m9plnt0538gc2wb4l5pzcgr80000gp/T/ipykernel_9098/4080900513.py:57: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S = float(row["Close"])
/var/folders/vb/m9plnt0538gc2wb4l5pzcgr80000gp/T/ipykernel_9098/4080900513.py:58: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma = float(row["forecast_vol"])
